In [16]:
#%% imports
import vedo as vd
import numpy as np
from vedo.pyplot import plot
from vedo import Latex
from matplotlib import pyplot as plt

vd.settings.default_backend = 'vtk'

In [17]:
#%% Callbacks
msg = vd.Text2D(pos='bottom-left', font="VictorMono")  # an empty text

#bounds of the mesh
x_min, x_max = 0, 3  
y_min, y_max = 0, 3  
z_min, z_max = 0, 1  
bounds = [(x_min, x_max), (y_min, y_max), (z_min, z_max)]

def OnMouseMove(evt):  # called every time mouse moves!
    global Xi
    if evt.picked3d is None:  # mouse hits nothing, return.
        return                       
    pt  = evt.picked3d  # 3d coords of point under mouse
    X   = np.array([pt[0], pt[1], objective([pt[0], pt[1]])])  # X = (x,y,e(x,y))
    Xi = np.append(Xi, [X], axis=0)  # append to the list of points
    if len(Xi) > 1:  # need at least two points to compute a distance
        txt =(
            f"X:  {vd.precision(X, 2)}\n"
            f"dX: {vd.precision(Xi[-1, 0:2] - Xi[-2, 0:2], 2)}\n"
            f"dE: {vd.precision(Xi[-1, 2] - Xi[-2, 2], 2)}\n"
        )
        ar = vd.Arrow(Xi[-2, :], Xi[-1, :], s=0.001, c='orange5')
        plt.add(ar)  # add the arrow
    else:
        txt = f"X: {vd.precision(X, 2)}"
    msg.text(txt)  # update text message
    c = vd.Cylinder([np.append(Xi[-1, 0:2], 0.0), Xi[-1, :]], r=0.01, c='orange5')
    plt.remove("Cylinder")    
    fp = fplt3d[0].flagpole(txt, point=X, s=0.08, c='k', font="Quikhand")
    fp.follow_camera()  # make it always face the camera
    plt.remove("FlagPole")  # remove the old flagpole
    plt.add(fp, c)  # add the new flagpole and new cylinder
    plt.render()  # re-render the scene

def OnLeftClick(evt):
    if evt.actor:
        click_pos = evt.picked3d
        if click_pos is not None:
            global Xi, prev_pathplot, pathplot
            # Convert clicked position to array with objective value
            new_point = np.array([click_pos[0], click_pos[1], objective([click_pos[0], click_pos[1]])])
            if Xi.size > 0:  # Check if Xi is not empty
                Xi = np.empty((0, 3))  # Clear the path points
            # Insert the new point into Xi
            Xi = np.append(Xi, [new_point], axis=0)
            # Remove all arrows
            plt.remove("Arrow").render()
            # Remove the previous path plot if it exists
            if prev_pathplot is not None:
                plt.remove(prev_pathplot).render()
            # Set the current optimization candidate to the clicked position
            global current_optimization_candidate
            current_optimization_candidate = click_pos[:2]
            print(f"Optimization candidate set to: {current_optimization_candidate}")
            # Update the plot with the new point
            update_path_plot()

def update_path_plot():
    global prev_pathplot, pathplot
    if len(Xi) > 1:  # need at least two points to compute a distance
        txt = (
            f"X:  {vd.precision(Xi[-1], 2)}\n"
            f"dX: {vd.precision(Xi[-1, 0:2] - Xi[-2, 0:2], 2)}\n"
            f"dE: {vd.precision(Xi[-1, 2] - Xi[-2, 2], 2)}\n"
        )
        ar = vd.Arrow(Xi[-2, :], Xi[-1, :], s=0.001, c='orange5')
        plt.add(ar)  # add the arrow to the plot
        if prev_pathplot is not None:
            plt.remove(prev_pathplot)
        pathplot = plot(Xi[:, 2], lw=3, c='orange5', title="Gradient descent", xtitle="Point", ytitle="Value")
        pathplot = pathplot.clone2d(pos='top-left')
        prev_pathplot = pathplot
        plt.add(pathplot)
    else:
        txt = f"X: {vd.precision(Xi[-1], 2)}"
    msg.text(txt)  # update text message
    c = vd.Cylinder([np.append(Xi[-1, 0:2], 0.0), Xi[-1, :]], r=0.01, c='orange5')
    plt.remove("Cylinder")
    fp = fplt3d[0].flagpole(txt, point=Xi[-1], s=0.08, c='k', font="Quikhand")
    fp.follow_camera()  # make it always face the camera
    plt.remove("FlagPole")  # remove the old flagpole
    plt.add(fp, c)  # add the new flagpole and new cylinder
    plt.render()  # re-render the scene

def OnKeyPress(evt):  ### called every time a key is pressed
    if evt.keypress in ['c', 'C']:  # reset Xi and the arrows
        global Xi
        Xi = np.empty((0, 3))
        plt.remove("Arrow").render()

def OnSliderAlpha(widget, event):  ### called every time the slider is moved
    val = widget.value  # get the slider value
    fplt3d[0].alpha(val)  # set the alpha (transparency) value of the surface
    fplt3d[1].alpha(val)  # set the alpha (transparency) value of the isolines

def OnButtonPress(widget, event):  ### Define the button callback function with the correct signature
    global current_optimization_candidate, Xi
    if Xi.size > 0:  # Ensure there is a point to perform gradient descent
        new_point = step(objective, current_optimization_candidate, gradient_direction, bounds)
        new_point = np.array([new_point[0], new_point[1], objective(new_point)])
        current_optimization_candidate = new_point[:2]
        Xi = np.append(Xi, [new_point], axis=0)
        update_path_plot()

def OnNewtonButtonPress(widget, event):  ### Define the button callback function with the correct signature
    global current_optimization_candidate, Xi
    if Xi.size > 0:  # Ensure there is a point to perform gradient descent
        new_point = step(objective, current_optimization_candidate, Newton_direction, bounds)
        new_point = np.array([new_point[0], new_point[1], objective(new_point)])
        current_optimization_candidate = new_point[:2]
        Xi = np.append(Xi, [new_point], axis=0)
        print(f"Xi updated with new point: {Xi}")  # Debug statement
        update_path_plot()

In [18]:
#%% Optimization functions

def gradient_fd(func, X, h=0.001):  # finite difference gradient
    x, y = X[0], X[1]
    gx = (func([x + h, y]) - func([x - h, y])) / (2 * h)
    gy = (func([x, y + h]) - func([x, y - h])) / (2 * h)
    return gx, gy

def Hessian_fd(func, X, h=0.001):  # finite difference Hessian
    x, y = X[0], X[1]
    gxx = (func([x + h, y]) - 2 * func([x, y]) + func([x - h, y])) / h ** 2
    gyy = (func([x, y + h]) - 2 * func([x, y]) + func([x, y - h])) / h ** 2
    gxy = (func([x + h, y + h]) - func([x + h, y - h]) - func([x - h, y + h]) + func([x - h, y - h])) / (4 * h ** 2)
    H = np.array([[gxx, gxy], [gxy, gyy]])
    return H

def gradient_direction(func, X):  # compute gradient step direction
    g = gradient_fd(func, X)
    return -np.array(g)

def Newton_direction(func, X):  # compute Newton step direction
    g = gradient_fd(func, X)
    H = Hessian_fd(func, X)
    d = -np.linalg.solve(H, np.array(g))
    return d

def objective(X):
    x, y = X[0], X[1]
    return np.sin(2 * x * y) * np.cos(3 * y) / 2 + 1 / 2

def boundary_check(X, bounds):
    X[0] = max(min(X[0], bounds[0][1]), bounds[0][0])
    X[1] = max(min(X[1], bounds[1][1]), bounds[1][0])
    if len(X) > 2:
        X[2] = max(min(X[2], bounds[2][1]), bounds[2][0])  # z-axis boundary check
    return X

def line_search(func, X, d):
    alpha = 0.5
    while func(X + d * alpha) > func(X):  # If the function value does not decrease, reduce alpha
        alpha *= 0.5  # by half and try again
    return alpha

def step(func, X, search_direction_function, bounds):
    d = search_direction_function(func, X)
    alpha = line_search(func, X, d)
    new_X = X + d * alpha
    new_X = boundary_check(new_X, bounds)
    return new_X

def optimize(func, X, search_direction_function, tol=1e-6, iter_max=10, bounds=None):
    for i in range(iter_max):
        X = step(func, X, search_direction_function, bounds)
        if np.linalg.norm(gradient_fd(func, X)) < tol:
            break
    return X

Xi = np.empty((0, 3))
# test the optimization functions
X = optimize(objective, [0.6, 0.6], Newton_direction, tol=1e-6, iter_max=100, bounds=bounds)
pathplot = None  # Initialize the path plot variable
prev_pathplot = None  # Initialize the previous path plot variable
current_optimization_candidate = np.array([0.6, 0.6])


In [19]:
#%% Plotting

plt = vd.Plotter(bg2='lightblue')  # Create the plotter
fplt3d = plot(lambda x, y: objective([x, y]), c='terrain')  # create a plot from the function e. fplt3d is a list containing surface mesh, isolines, and axis
fplt2d = fplt3d.clone()  # clone the plot to create a 2D plot
fplt2d[0].lighting('off')  # turn off lighting for the 2D plot
fplt2d[0].vertices[:, 2] = 0  # set the z-coordinate of the mesh to 0
fplt2d[1].vertices[:, 2] = 0  # set the z-coordinate of the isolines to 0
#plt.add_callback('mouse move', OnMouseMove)  # add Mouse move callback
plt.add_callback('key press', OnKeyPress)  # add Keyboard callback
plt.add_slider(OnSliderAlpha, 0., 1., 1., title="Alpha")  # add a slider for the alpha value of the surface
plt.add_callback('LeftButtonPressEvent', OnLeftClick)  # add left mouse button click callback
plt.add_button(OnButtonPress, pos=(0.8, 0.9), states=["Gradient Descent Step"], size=20, c="w", bc="blue")
plt.add_button(OnNewtonButtonPress, pos=(0.8, 0.8), states=["Newton Step"], size=20, c="w", bc="red")
plt.show([fplt3d, fplt2d], msg, __doc__, viewup='z')
plt.close()

Optimization candidate set to: [0.53036435 1.76387856]
Xi updated with new point: [[0.53036435 1.76387856 0.76145637]
 [0.53036435 1.76387856 0.76145637]]
Xi updated with new point: [[0.53036435 1.76387856 0.76145637]
 [0.53036435 1.76387856 0.76145637]
 [0.53036435 1.76387856 0.76145637]]


[vedoplot] WARNING: Null range for Figure Gradient descent... returning an empty Assembly.
[vedoplot] WARNING: Null range for Figure Gradient descent... returning an empty Assembly.


Xi updated with new point: [[0.53036435 1.76387856 0.76145637]
 [0.53036435 1.76387856 0.76145637]
 [0.53036435 1.76387856 0.76145637]
 [0.53036435 1.76387856 0.76145637]]
Xi updated with new point: [[0.53036435 1.76387856 0.76145637]
 [0.53036435 1.76387856 0.76145637]
 [0.53036435 1.76387856 0.76145637]
 [0.53036435 1.76387856 0.76145637]
 [0.53036435 1.76387856 0.76145637]]
Xi updated with new point: [[0.53036435 1.76387856 0.76145637]
 [0.53036435 1.76387856 0.76145637]
 [0.53036435 1.76387856 0.76145637]
 [0.53036435 1.76387856 0.76145637]
 [0.53036435 1.76387856 0.76145637]
 [0.53036435 1.76387856 0.76145637]]


[vedoplot] WARNING: Null range for Figure Gradient descent... returning an empty Assembly.
[vedoplot] WARNING: Null range for Figure Gradient descent... returning an empty Assembly.
[vedoplot] WARNING: Null range for Figure Gradient descent... returning an empty Assembly.


Xi updated with new point: [[0.53036435 1.76387856 0.76145637]
 [0.53036435 1.76387856 0.76145637]
 [0.53036435 1.76387856 0.76145637]
 [0.53036435 1.76387856 0.76145637]
 [0.53036435 1.76387856 0.76145637]
 [0.53036435 1.76387856 0.76145637]
 [0.53036435 1.76387856 0.76145637]]
Xi updated with new point: [[0.53036435 1.76387856 0.76145637]
 [0.53036435 1.76387856 0.76145637]
 [0.53036435 1.76387856 0.76145637]
 [0.53036435 1.76387856 0.76145637]
 [0.53036435 1.76387856 0.76145637]
 [0.53036435 1.76387856 0.76145637]
 [0.53036435 1.76387856 0.76145637]
 [0.53036435 1.76387856 0.76145637]]
Xi updated with new point: [[0.53036435 1.76387856 0.76145637]
 [0.53036435 1.76387856 0.76145637]
 [0.53036435 1.76387856 0.76145637]
 [0.53036435 1.76387856 0.76145637]
 [0.53036435 1.76387856 0.76145637]
 [0.53036435 1.76387856 0.76145637]
 [0.53036435 1.76387856 0.76145637]
 [0.53036435 1.76387856 0.76145637]
 [0.53036435 1.76387856 0.76145637]]


...

Xi updated with new point: [[1.12393411 2.596721   0.48624209]
 [1.12393411 2.596721   0.48624209]
 [1.12393411 2.596721   0.48624209]
 [1.12393411 2.596721   0.48624209]
 [1.12393411 2.596721   0.48624209]
 [1.12393411 2.596721   0.48624209]
 [1.12393411 2.596721   0.48624209]]
Xi updated with new point: [[1.12393411 2.596721   0.48624209]
 [1.12393411 2.596721   0.48624209]
 [1.12393411 2.596721   0.48624209]
 [1.12393411 2.596721   0.48624209]
 [1.12393411 2.596721   0.48624209]
 [1.12393411 2.596721   0.48624209]
 [1.12393411 2.596721   0.48624209]
 [1.12393411 2.596721   0.48624209]]
Xi updated with new point: [[1.12393411 2.596721   0.48624209]
 [1.12393411 2.596721   0.48624209]
 [1.12393411 2.596721   0.48624209]
 [1.12393411 2.596721   0.48624209]
 [1.12393411 2.596721   0.48624209]
 [1.12393411 2.596721   0.48624209]
 [1.12393411 2.596721   0.48624209]
 [1.12393411 2.596721   0.48624209]
 [1.12393411 2.596721   0.48624209]]


[vedoplot] WARNING: Null range for Figure Gradient descent... returning an empty Assembly.
[vedoplot] WARNING: Null range for Figure Gradient descent... returning an empty Assembly.


Xi updated with new point: [[1.12393411 2.596721   0.48624209]
 [1.12393411 2.596721   0.48624209]
 [1.12393411 2.596721   0.48624209]
 [1.12393411 2.596721   0.48624209]
 [1.12393411 2.596721   0.48624209]
 [1.12393411 2.596721   0.48624209]
 [1.12393411 2.596721   0.48624209]
 [1.12393411 2.596721   0.48624209]
 [1.12393411 2.596721   0.48624209]
 [1.12393411 2.596721   0.48624209]]
Xi updated with new point: [[1.12393411 2.596721   0.48624209]
 [1.12393411 2.596721   0.48624209]
 [1.12393411 2.596721   0.48624209]
 [1.12393411 2.596721   0.48624209]
 [1.12393411 2.596721   0.48624209]
 [1.12393411 2.596721   0.48624209]
 [1.12393411 2.596721   0.48624209]
 [1.12393411 2.596721   0.48624209]
 [1.12393411 2.596721   0.48624209]
 [1.12393411 2.596721   0.48624209]
 [1.12393411 2.596721   0.48624209]]


[vedoplot] WARNING: Null range for Figure Gradient descent... returning an empty Assembly.
[vedoplot] WARNING: Null range for Figure Gradient descent... returning an empty Assembly.
[vedoplot] WARNING: Null range for Figure Gradient descent... returning an empty Assembly.


Optimization candidate set to: [1.80446464 1.96399844]
Optimization candidate set to: [0.51675585 1.76851853]
Xi updated with new point: [[0.51675585 1.76851853 0.7703161 ]
 [0.64238914 1.20377095 0.05428325]
 [0.64888726 1.03755001 0.01272807]
 [0.70655461 1.08419535 0.00344898]
 [0.7169797  1.04948331 0.00109638]
 [0.73425174 1.05871719 0.00036304]
 [0.7427512  1.03870118 0.00035559]
 [0.74996773 1.0534152  0.00010841]
 [0.74752994 1.04468551 0.00003413]
 [0.74986182 1.04917999 0.00001064]
 [0.74915791 1.04644668 0.00000335]
 [0.74957969 1.04682222 0.00000084]]
Xi updated with new point: [[0.51675585 1.76851853 0.7703161 ]
 [0.64238914 1.20377095 0.05428325]
 [0.64888726 1.03755001 0.01272807]
 [0.70655461 1.08419535 0.00344898]
 [0.7169797  1.04948331 0.00109638]
 [0.73425174 1.05871719 0.00036304]
 [0.7427512  1.03870118 0.00035559]
 [0.74996773 1.0534152  0.00010841]
 [0.74752994 1.04468551 0.00003413]
 [0.74986182 1.04917999 0.00001064]
 [0.74915791 1.04644668 0.00000335]
 [0.749